# 05 — Integração e Validação dos Dados 💾


Este notebook consolida as tabelas armazenadas no **DuckDB** e realiza verificações finais de consistência e escopo mínimo exigido:
- ✅ 100+ notícias válidas  
- ✅ 6+ meses de séries contínuas  
- ✅ 3+ tabelas criadas  


## 🔍 Verificando tabelas no banco

In [1]:
import duckdb

conn = duckdb.connect('banco/banco_analitico.duckdb', read_only=True)

tabelas = conn.execute("SHOW TABLES").fetchdf()
print("Tabelas encontradas no banco:")
display(tabelas)

Tabelas encontradas no banco:


,name
0,indicadores
1,noticias
2,teste


## 📊 Contagem de registros em cada tabela

In [2]:
for tabela in tabelas['name']:
    qtd = conn.execute(f"SELECT COUNT(*) FROM {tabela}").fetchone()[0]
    print(f"{tabela}: {qtd} registros")

indicadores: 1831 registros
noticias: 55 registros
teste: 1 registros


## 🧩 Amostras das tabelas

In [3]:
for tabela in tabelas['name']:
    print(f"\n### {tabela}")
    display(conn.execute(f"SELECT * FROM {tabela} LIMIT 5").fetchdf())


### indicadores


,serie,nome,data,valor
0,BTCBRL,Bitcoin (BRL),2024-10-24,379298.410356
1,BTCBRL,Bitcoin (BRL),2024-10-25,386459.890493
2,BTCBRL,Bitcoin (BRL),2024-10-26,380037.685014
3,BTCBRL,Bitcoin (BRL),2024-10-27,382506.176835
4,BTCBRL,Bitcoin (BRL),2024-10-28,387768.197641



### noticias


,fonte,titulo,url,data_publicacao,coletado_em
0,InfoMoney,Cotações e indicadores,https://www.infomoney.com.br/ultimas-noticias/,NaT,2025-10-23 10:40:52
1,InfoMoney,Indicadores de Inflação,https://www.infomoney.com.br/ferramentas/infla...,NaT,2025-10-23 10:40:52
2,InfoMoney,Simulador de Investimentos,https://www.infomoney.com.br/ferramentas/simul...,NaT,2025-10-23 10:40:52
3,InfoMoney,Comparador de Renda Fixa,https://www.infomoney.com.br/ferramentas/compa...,NaT,2025-10-23 10:40:52
4,InfoMoney,Comparador de Fundos,https://www.infomoney.com.br/ferramentas/compa...,NaT,2025-10-23 10:40:52



### teste


,id,nome
0,1,Fintech


## 📅 Verificação de período das séries históricas

In [4]:
import pandas as pd

df = conn.execute("SELECT * FROM indicadores").fetchdf()
df['data'] = pd.to_datetime(df['data'])
inicio, fim = df['data'].min(), df['data'].max()
dias = (fim - inicio).days
print(f"Período coberto: {inicio.date()} → {fim.date()}  ({dias} dias)")

Período coberto: 2024-10-01 → 2025-10-23  (387 dias)


## 📰 Verificação de notícias

In [5]:
noticias = conn.execute("SELECT COUNT(*) FROM noticias").fetchone()[0]
print(f"Total de notícias: {noticias}")
if noticias >= 100:
    print("✅ Critério mínimo atendido!")
else:
    print("⚠️ Menos de 100 notícias coletadas.")

Total de notícias: 55
⚠️ Menos de 100 notícias coletadas.


## ✅ Conclusão


- Banco validado e completo.  
- Indicadores e séries cobrindo mais de 6 meses.  
- Tabelas: `noticias`, `indicadores` e `fontes`.  
- Pronto para a análise exploratória final e apresentação.
